# Assignment

In this assignment we will train a convolutional neural network (CNN) on the CIFAR-10 dataset.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 645kB 15.1MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [3]:
# --- Download dataset
datasets.download(name='cifar')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='cifar')
inputs = client.get_inputs(Input)

[ 2021-04-19 09:51:26 ] [====================] 100.000% : Iterating | 000001    

In [4]:
# --- Yield one example
xs, ys = next(gen_train)

# --- Print dict keys
print('xs keys: {}'.format(xs.keys()))
print('ys keys: {}'.format(ys.keys()))

xs keys: dict_keys(['dat'])
ys keys: dict_keys(['class'])


In [5]:
xs, ys = next(gen_train)
print('xs shape: {}'.format(xs['dat'].shape))
print('ys shape: {}'.format(ys['class'].shape))

xs shape: (32, 32, 32, 3)
ys shape: (32, 1)


# Training

In this assignment we will train a basic convolutional neural network on the CIFAR-10 dataset. At minumum you must include the following baseline techniques covered in the tutorial:

* convolutional operations
* batch normalization
* activation function
* subsampling

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [6]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (3, 3),
    'padding': 'same'}

In [7]:
# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

In [8]:

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))

l1 = conv1(16, inputs['dat'])
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))

f0 = layers.Flatten()(l4)
# --- Define model
# ...
# ...

logits = {}
logits['class'] = layers.Dense(10, name='class')(f0)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [9]:
# --- Compile model

model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4), 
    loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'class': 'sparse_categorical_accuracy'})

### Train the model

In [10]:
model.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

Epoch 1/12
250/250 [==============================] - 39s 25ms/step - loss: 2.3203 - sparse_categorical_accuracy: 0.1796
Epoch 2/12
250/250 [==============================] - 6s 25ms/step - loss: 1.7994 - sparse_categorical_accuracy: 0.3593
Epoch 3/12
250/250 [==============================] - 6s 25ms/step - loss: 1.6754 - sparse_categorical_accuracy: 0.4022
Epoch 4/12
250/250 [==============================] - 12s 48ms/step - loss: 1.5434 - sparse_categorical_accuracy: 0.4347 - val_loss: 1.5041 - val_sparse_categorical_accuracy: 0.4613
Epoch 5/12
250/250 [==============================] - 6s 25ms/step - loss: 1.4900 - sparse_categorical_accuracy: 0.4695
Epoch 6/12
250/250 [==============================] - 6s 25ms/step - loss: 1.4523 - sparse_categorical_accuracy: 0.4850
Epoch 7/12
250/250 [==============================] - 6s 25ms/step - loss: 1.3497 - sparse_categorical_accuracy: 0.5178
Epoch 8/12
250/250 [==============================] - 12s 47ms/step - loss: 1.3268 - sparse_categ

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

In [11]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

# --- Aggregate all examples
xs = []
ys = []

for x, y in test_valid:
    xs.append(x['dat'])
    ys.append(y['class'])

xs = np.concatenate(xs)
ys = np.concatenate(ys)

# --- Predict
logits = model.predict(xs)

if type(logits) is dict:
    logits = logits['class']

# --- Argmax
pred = np.argmax(logits, axis=1)


[ 2021-04-19 09:53:54 ] [====================] 100.000% : Iterating | 012000    

In [12]:
df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = ys[:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())

0.5538333333333333


**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [13]:
#from google.colab import drive
#path=drive.mount('/gdrive', force_remount=True)
# --- Create *.csv

                              
# --- Serialize *.csv
#fname = 'Fnaghman/models/cnn/results.csv'.format(path)
os.makedirs(os.path.dirname('./'), exist_ok=True)
df.to_csv('./cnn.csv')

# Submission

Use the following line to save your model for submission:

In [14]:
model.save('./cnn.hdf5')

del model
model = models.load_model('./cnn.hdf5', compile=False)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.